In [79]:
# import numpy as np
# import torch
# # from transformers import BertTokenizer, BertForMaskedLM
# from pytorch_pretrained_bert import BertTokenizer, BertForMaskedLM
# import re
# from random import *


# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')      #导入词库
# bert = BertForMaskedLM.from_pretrained('bert-base-uncased')       #导入模型


# bert.eval()
# # bert.to('cuda:0')       #移动到GPU，没有GPU就不要这行代码了


# def ques_proce(file):
#     f = open(file, 'r', encoding='gb18030', errors='ignore')
#     buffer = f.readline()
#     choices=[]
#     while buffer!='':
#         list=buffer.split()
#         one_que=[list[idx] for idx in [2,4,6,8]]
#         choices.append(one_que)
#         buffer=f.readline()
#     return choices
    
# #选项提取
# choices=ques_proce('question.txt')          #处理问题（选项）文本
# choices_idx=[]
# for choice in choices:          #进行tokenize化
#     choice_idx=tokenizer.convert_tokens_to_ids(choice)
#     choices_idx.append(choice_idx)

from pytorch_pretrained_bert import BertTokenizer

# 加载 BERT Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def ques_proce(file):
    f = open(file, 'r', encoding='gb18030', errors='ignore')
    buffer = f.readline()
    choices = []
    while buffer != '':
        list = buffer.strip().split()
        if len(list) >= 9:
            one_que = [list[idx] for idx in [2, 4, 6, 8]]
            choices.append(one_que)
        else:
            print(f"Skipping incomplete line: {buffer.strip()}")
        buffer = f.readline()

    return choices

# 清理选项，确保没有点号
def clean_choices(choices):
    cleaned_choices = []
    for choice in choices:
        cleaned_choice = [item.rstrip('.') for item in choice]
        cleaned_choices.append(cleaned_choice)
    return cleaned_choices

# Tokenizing step with fallback for out of vocabulary words
def safe_tokenize(choice, tokenizer):
    try:
        # 尝试直接转换为ID
        return tokenizer.convert_tokens_to_ids(choice)
    except KeyError as e:
        print(f"Warning: Token '{e}' not found in vocab.")
        # 对每个词进行 tokenization，拆分词
        return [tokenizer.convert_tokens_to_ids(token) for token in tokenizer.tokenize(choice)]

# 主代码执行部分
choices = ques_proce('question.txt')
cleaned_choices = clean_choices(choices)

choices_idx = []
for choice in cleaned_choices:
    choice_idx = []
    for token in choice:
        # Tokenize the choice one token at a time
        token_ids = safe_tokenize(token, tokenizer)
        choice_idx.extend(token_ids)  # 把所有 token 的 ID 连接起来
    choices_idx.append(choice_idx)

print(choices_idx)


#建立预测概率矩阵
ans_prob=[]
for i in range(len(choices)):
    ans_prob.append([0.0,0.0,0.0,0.0])

def sen2maskIdx(sen_list):
    now_mask=0
    result=[]
    for idx in range(len(sen_list)):
        sen=sen_list[idx]
        mask_num=sen.count('[MASK]')
        maskIdx=[i+now_mask for i in range(mask_num)]
        now_mask+=mask_num
        result.append(maskIdx)
    return result
 
 
 
def pass_proce(file, per_times):
    """
    处理文本文件，将含有 [MASK] 的句子与其他句子组合生成新的句子对。
    
    参数：
        file (str): 文本文件路径。
        per_times (int): 每个句子需要与多少个不同的句子组合。
    
    返回：
        list: 每个句子组合生成的句子对和掩码索引。
    """
    try:
        # 读取和处理文本内容
        with open(file, 'r', encoding='gb18030', errors='ignore') as f:
            buffer = f.read()
        
        buffer = re.sub(r'\n', ' ', buffer)  # 替换换行符为空格
        buffer = re.sub(r'\(\d{1,2}\)_{1,9}', '[MASK]', buffer)  # 替换特定模式为 [MASK]
        sen_list = buffer.split('.')  # 以句号分割成句子列表

        # 确保句子列表非空
        if not sen_list or all(len(sen.strip()) == 0 for sen in sen_list):
            raise ValueError("文本内容为空，无法生成有效句子。")

        for_all_mask = []
        sen2maskidx = sen2maskIdx(sen_list)  # 假定该函数正确生成句子到掩码索引的映射

        # 遍历句子
        for sen in sen_list:
            if '[MASK]' in sen:  # 只处理包含 [MASK] 的句子
                for_this_mask = []

                for i in range(per_times):
                    ano_idx = randint(0, len(sen_list) - 1)

                    # 确保选择的句子与当前句子不重复
                    while sen_list[ano_idx] == sen:
                        ano_idx = randint(0, len(sen_list) - 1)

                    # 根据索引顺序构造句子对和掩码索引
                    if sen_list.index(sen) > ano_idx:
                        temp_sen = '[CLS]' + sen_list[ano_idx].strip() + ' [SEP] ' + sen.strip() + ' [SEP]'
                        mask_idx = sen2maskidx[ano_idx] + sen2maskidx[sen_list.index(sen)]
                    else:
                        temp_sen = '[CLS]' + sen.strip() + ' [SEP] ' + sen_list[ano_idx].strip() + ' [SEP]'
                        mask_idx = sen2maskidx[sen_list.index(sen)] + sen2maskidx[ano_idx]

                    for_this_mask.append((temp_sen, mask_idx))  # 添加生成的句子对和掩码索引

                for_all_mask.append(for_this_mask)

        return for_all_mask

    except FileNotFoundError:
        raise FileNotFoundError(f"无法找到文件: {file}")
    except Exception as e:
        raise RuntimeError(f"处理文件时出错: {str(e)}")
        
#文本处理
text=pass_proce("passage.txt",17)   #处理文章文本
for mask_sen in text:
    for per_sen in mask_sen:
        tokenized_text = tokenizer.tokenize(per_sen[0])
        broke_point=tokenized_text.index('[SEP]')
        segments_ids=[0]*(broke_point+1)+[1]*(len(tokenized_text)-broke_point-1)
        que_idxs=per_sen[1]
 
        ids = torch.tensor([tokenizer.convert_tokens_to_ids(tokenized_text)])
        segments_tensors = torch.tensor([segments_ids])
        ids = torch.tensor([tokenizer.convert_tokens_to_ids(tokenized_text)])
        segments_tensors = torch.tensor([segments_ids])
         
        #mask的位置提取
        mask_num=tokenized_text.count('[MASK]')
        mask_idxs=[idx for idx in range(len(tokenized_text)) if tokenized_text[idx]=='[MASK]']
 
 
        #预测答案
        result = bert(ids,segments_tensors)
        for i in range(mask_num):
            mask_idx=mask_idxs[i]
            this_ans_prob = [result[0][mask_idx][choice_idx] for choice_idx in choices_idx[que_idxs[i]]]
            ans_prob[que_idxs[i]]=[ans_prob[que_idxs[i]][j]+this_ans_prob[j] for j in range(4)]

#归一化
for i in range(len(choices)):
    for j in range(4):
        ans_prob[i][j]/=17

#计算预测答案

print(ans_prob)

ans_pred = []
for per_que in ans_prob:
    # 确保per_que被正确处理
    if isinstance(per_que, torch.Tensor):
        # 先detach，确保与计算图分离，再转为CPU的NumPy数组
        per_que = per_que.detach().cpu().numpy()
    elif isinstance(per_que, list):
        # 如果是列表，直接转为NumPy数组
        per_que = np.array([pq.detach().cpu().numpy() if isinstance(pq, torch.Tensor) else pq for pq in per_que])

    # 获取最大值的索引
    ans = ['A', 'B', 'C', 'D'][per_que.argmax(axis=0)]
    ans_pred.append(ans)

print(ans_pred)





def ans_proce(file):
    f = open(file, 'r', encoding='gb18030', errors='ignore')
    buffer = f.readline()
    answers = []
    while buffer!='':
        buffer=buffer.strip()
        answers.append(buffer)
        buffer=f.readline()
    # print(answers)
    return answers
 
 
#导入正确答案
ans_conrrect=ans_proce('answer.txt')
 
 
 
#计算正确率
correct=0.0
for i in range(len(choices)):
    if ans_pred[i]==ans_conrrect[i]:
        correct+=1
print("the correct rate is :"+str(correct/len(choices)*100.0)+"%")



[[1037, 1040, 1046, 1057, 1055, 1056, 1041, 1040, 1052, 1054, 1051, 1049, 1051, 1056, 1041, 1040, 1037, 1039, 1044, 1045, 1041, 1058, 1041, 1040, 1041, 1060, 1052, 1041, 1054, 1045, 1041, 1050, 1039, 1041, 1040], [1059, 1051, 1054, 1047, 1055, 1039, 1044, 1051, 1051, 1048, 1039, 1051, 1057, 1054, 1056, 1052, 1054, 1041, 1055, 1055], [1042, 1037, 1058, 1051, 1054, 1041, 1040, 1056, 1051, 1048, 1041, 1054, 1037, 1056, 1041, 1040, 1056, 1054, 1057, 1055, 1056, 1041, 1040, 1037, 1039, 1047, 1050, 1051, 1059, 1048, 1041, 1040, 1043, 1041, 1040], [1043, 1037, 1054, 1040, 1041, 1050, 1045, 1050, 1043, 1044, 1051, 1049, 1041, 1059, 1051, 1054, 1047, 1038, 1057, 1055, 1045, 1050, 1041, 1055, 1055, 1044, 1051, 1057, 1055, 1041, 1059, 1051, 1054, 1047], [1041, 1060, 1041, 1054, 1039, 1045, 1055, 1041, 1055, 1056, 1057, 1040, 1061, 1041, 1060, 1052, 1048, 1051, 1054, 1041, 1056, 1041, 1037, 1039, 1044], [1042, 1051, 1051, 1040, 1043, 1057, 1041, 1055, 1056, 1055, 1048, 1041, 1055, 1055, 1051, 1050